In [162]:
import scipy.io
import pandas as pd

import os
import time

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
from scipy import stats
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional

In [163]:
mat = scipy.io.loadmat('Oxford_Battery_Degradation_Dataset_1.mat')

In [164]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Cell1', 'Cell2', 'Cell3', 'Cell4', 'Cell5', 'Cell6', 'Cell7', 'Cell8'])

In [165]:
input_data = [[], []]

In [166]:
for i in range(0, 83):
    if (i < 10):
        if (i == 0):
            temp = "cyc0000"
        else:
            temp = "cyc0" + str(i) + "00"
    else:
        temp = "cyc" + str(i) + "00"
    try:
        curr = mat["Cell1"][0][temp][0][0]["C1ch"][0][0]['q'][0][-1][0]
    except ValueError:
        curr = float("NaN")
    input_data[0].append(i)
    input_data[1].append(curr)
    print(curr)

724.1208914093187
714.5350426825697
709.8543464961379
707.0393485194566
702.2342863457235
698.8440013150872
695.4850715278354
691.6932258285015
687.6300422440017
684.2488065128581
681.0607272448832
677.8486336050008
674.7281887527703
671.556081370731
668.6709791990864
nan
662.9724287937147
nan
656.4621886431951
653.2751603988444
650.1157422984008
647.1105116933119
644.0549444561768
640.7948747304991
638.1391374235501
635.3790871886936
632.3887851445143
629.9572173218418
627.3439847566834
624.1095617381898
622.448908879052
619.9778349196439
617.3031788485108
615.3476618618469
nan
611.340570310974
609.7195455723199
605.1998563440754
604.1349210491537
602.8723865926853
600.9075598059693
598.2787878589917
596.7047192299974
594.08713143121
591.9284795739374
589.3588371850756
588.0833074787903
nan
584.2617834340922
nan
579.5968326875441
577.6717759357741
575.5147033984103
574.7467034891866
572.8103356272312
568.9614464139573
568.632337365943
566.7898372362026
564.7710880962863
563.0229675191

In [167]:
df_input = pd.DataFrame(input_data).transpose()
df_input = df_input.rename(columns = {0: "cycle", 1:"q_value"})
df_input

,cycle,q_value
0,0.0,724.120891
1,1.0,714.535043
2,2.0,709.854346
3,3.0,707.039349
4,4.0,702.234286
...,...,...
78,78.0,534.715467
79,79.0,534.731729
80,80.0,533.296215
81,81.0,529.457138


In [168]:
df_input['interpolate_spline'] = df_input['q_value'].interpolate(option='spline')
df_input

,cycle,q_value,interpolate_spline
0,0.0,724.120891,724.120891
1,1.0,714.535043,714.535043
2,2.0,709.854346,709.854346
3,3.0,707.039349,707.039349
4,4.0,702.234286,702.234286
...,...,...,...
78,78.0,534.715467,534.715467
79,79.0,534.731729,534.731729
80,80.0,533.296215,533.296215
81,81.0,529.457138,529.457138


In [169]:
# Split train data and test data

train_size = int(len(df_input)*0.8)

train_data = df_input.iloc[:train_size]
test_data = df_input.iloc[train_size:]

In [170]:
scaler = MinMaxScaler().fit(train_data)
train_scaled = scaler.transform(train_data)
test_scaled = scaler.transform(test_data)

In [171]:
# Create input dataset
def create_dataset (X, look_back = 1):
    Xs, ys = [], []

    for i in range(len(X)-look_back):
        v = X[i:i+look_back]
        Xs.append(v)
        ys.append(X[i+look_back])

    return np.array(Xs), np.array(ys)
LOOK_BACK = 30
X_train, y_train = create_dataset(train_scaled,LOOK_BACK)
X_test, y_test = create_dataset(test_scaled,LOOK_BACK)
# Print data shape
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (36, 30, 3)
y_train.shape:  (36, 3)
X_test.shape:  (0,)
y_test.shape:  (0,)


In [172]:
# Create GRU model
def create_gru(units):
    model = Sequential()
    # Input layer
    model.add(GRU (units = units, return_sequences = True,
    input_shape = [X_train.shape[1], X_train.shape[2]]))
    model.add(Dropout(0.2))
    # Hidden layer
    model.add(GRU(units = units))
    model.add(Dropout(0.2))
    model.add(Dense(units = 1))
    #Compile model
    model.compile(optimizer='adam',loss='mse')
    return model
model_gru = create_gru(64)

In [173]:
def fit_model(model):
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                               patience = 10)
    history = model.fit(X_train, y_train, epochs = 100,
                        validation_split = 0.2,
                        batch_size = 16, shuffle = False,
                        callbacks = [early_stop])
    return history
history_gru = fit_model(model_gru)

Epoch 1/100
2/2 [==============================] - 8s 1s/step - loss: nan - val_loss: nan
Epoch 2/100
2/2 [==============================] - 0s 66ms/step - loss: nan - val_loss: nan
Epoch 3/100
2/2 [==============================] - 0s 72ms/step - loss: nan - val_loss: nan
Epoch 4/100
2/2 [==============================] - 0s 64ms/step - loss: nan - val_loss: nan
Epoch 5/100
2/2 [==============================] - 0s 64ms/step - loss: nan - val_loss: nan
Epoch 6/100
2/2 [==============================] - 0s 73ms/step - loss: nan - val_loss: nan
Epoch 7/100
2/2 [==============================] - 0s 72ms/step - loss: nan - val_loss: nan
Epoch 8/100
2/2 [==============================] - 0s 72ms/step - loss: nan - val_loss: nan
Epoch 9/100
2/2 [==============================] - 0s 77ms/step - loss: nan - val_loss: nan
Epoch 10/100
2/2 [==============================] - 0s 88ms/step - loss: nan - val_loss: nan


In [174]:
y_test = scaler.inverse_transform(y_test)
y_train = scaler.inverse_transform(y_train)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
def plot_loss (history, model_name):
    plt.figure(figsize = (10, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Train vs Validation Loss for ' + model_name)
    plt.ylabel('Loss')
    plt.xlabel('epoch')
    plt.legend(['Train loss', 'Validation loss'], loc='upper right')

plot_loss (history_gru, 'GRU')